## Compute Atmospheric Relative Angular Momentum Forecast from CFSv2 Data
###### Inputs: CFSv2 1 deg lat/lon grib2 data obtained from NCEP NOMADS; need u-wind at all pressure levels 1000-1 hPa			
###### Outputs: Global Relative Atmospheric Angular Momentum Standardized Anomaly Forecast
##### Created by: Dr. Victor Gensini (Spring 2017) | weather.cod.edu/~vgensini								

In [111]:
#Import neccessary Python libraries (this example uses Python 2.7)
import numpy as np
import math, pygrib
import matplotlib.pyplot as plt
import pandas as pd

In [112]:
#Declare physical constants
pi = math.pi #Pi
rad = pi/180. #radians
gravity = 9.81 #gravitational acceleration ms^-2
earth_r = 6371220. #Earth's radius in (m)

### Open a CFSv2 grib2 forecast file
##### Data for this example obtained from: http://nomads.ncep.noaa.gov/pub/data/nccf/com/cfs/prod/cfs/

In [113]:
data_file = '/home/vgensini/data/cfsv2.grb2'

In [114]:
gr = pygrib.open(data_file) #Read CFSR grib2 file containing u-winds at all pressure levels into memory

### Read in all U-component (i.e., zonal) wind values

In [115]:
year = 2017 #1979-2010
month = 5 #1-12
day = 20 #1-28/29/30/31
hour = 0 #0,6,12,18

In [116]:
umsgs = gr.select(name='U component of wind',typeOfLevel='isobaricInhPa')

In [117]:
lats,lons = umsgs[0].latlons() #Read the lat/lon information from the u-wind grib2 message

In [118]:
uwnd = np.zeros((181, 360, 37)) #Could do some fancy dynamic shape finding here
for i,levs in enumerate(umsgs): #CFSv2 1.0 deg data is x-360,y-181,z-37
    uwnd[:,:,i] = levs.values   #Read zonal wind into a 3-D NumPy array

In [119]:
#dp is difference between CFSR pressure levels in Pa; set lowest and highest dp to zero as they are at integral bounds
#Vertical level information can be found here: http://rda.ucar.edu/datasets/ds093.0/#metadata/detailed.html?_do=y
dps=[0.,100.,200.,200.,300.,1000.,1000.,2000.,2000.,3000.,2500.,2500.,2500.,2500.,2500.,2500.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,5000.,2500.,2500.,2500.,2500.,2500.,2500.,2500.,2500.,2500.,2500.,0.]
dps = np.tile(np.array(dps),(181,360,1)) #Create a 3-D NumPy array of dp values

### Calculations

$$M_R=\frac{a^3}g\int\limits_{-\frac\pi2}^{\frac\pi2}\int\limits_0^{2\pi}\int\limits_1^{1000}\cos^2\phi{d}\phi{d}\lambda{udp}$$

In [120]:
UDP = np.multiply(uwnd,dps) #Calculate UDP
level_UDP = np.sum(UDP, axis=(2)) #Sum across all levels
zonalavg_UDP = np.mean(level_UDP, axis=(1)) #Zonal mean across all longitudes
dlat = 1.0 * rad #Latitude delta in radians (1.0 deg lat/lon grid spacing for this dataset)
aam = zonalavg_UDP * np.cos(lats[:,0] * rad) * np.cos(lats[:,0] * rad) * dlat * 2.* pi * earth_r**3 / gravity #Calculate AAM

### Result
##### aam represents the atmosphere's relative angular momentum by latitude (181 values in this example)

In [121]:
fcst_aam = np.sum(aam) #Sum all latitudes to get total forecast AAM

### Read in climatology data

In [122]:
df = pd.read_csv('/home/vgensini/data/aam_climo.csv')
format = '%Y-%m-%d-%H'
df['date']=pd.to_datetime(df['date'], format=format)
df=df.set_index(pd.DatetimeIndex(df['date']))
df['yr'], df['mn'], df['dy'], df['hr'] = df['date'].dt.year, df['date'].dt.month, df['date'].dt.day, df['date'].dt.hour
climo_aam = df.aam[(df.mn==month) & (df.dy==day) & (df.hr==hour)].mean()
stdev_climo_aam = df.aam[(df.mn==month) & (df.dy==day) & (df.hr==hour)].std()

### Calculate the standardized anomaly

In [123]:
print (fcst_aam-climo_aam)/stdev_climo_aam

0.334064381738
